In [ ]:
from pyspark.sql.functions import col, expr


df = (
    spark.readStream.format("delta").option("ignoreDeletes", "true")
    .table("dev_appian_poc.00_bronze.appian_raw_documents_ingest")
    .withColumn("parsed_document", expr("ai_parse_document(content)"))
    .select(
        col("parsed_document"),
        col("path"),
        col("modificationTime").alias("updatedTime"),
    )
)

query = (
    df.writeStream.format("delta")
    .option("checkpointLocation", "/Volumes/dev_appian_poc/00_bronze/checkpoints")
    .outputMode("append")
    .trigger(availableNow=True)
    .toTable("dev_appian_poc.01_silver.parsed_documents")
)

query.awaitTermination()